In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras

In [4]:
df=pd.read_csv('text.csv')

In [5]:
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [6]:
df.shape

(416809, 3)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [8]:
df.columns

Index(['Unnamed: 0', 'text', 'label'], dtype='object')

In [9]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [10]:
df.describe()

,label
count,416809.000000
mean,1.554271
std,1.490453
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [11]:
df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [13]:
emotion_dict = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

In [21]:
# df['length'] = df['text'].apply(len)

In [23]:
print(f"Rating value count: \n{df['label'].value_counts()}")

Rating value count: 
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


In [26]:
print(f"Label value count - percentage distribution: \n{round(df['label'].value_counts()/df.shape[0]*100,2)}")

Label value count - percentage distribution: 
label
1    33.84
0    29.07
3    13.75
4    11.45
2     8.29
5     3.59
Name: count, dtype: float64


In [28]:
data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)

/var/folders/8s/589js3cj3k5gf6dtzp4t6v7r0000gn/T/ipykernel_94889/964725730.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)


In [30]:
print(f"Rating value count: \n{data2['label'].value_counts()}")

Rating value count: 
label
0    14000
1    14000
2    14000
3    14000
4    14000
5    14000
Name: count, dtype: int64


In [32]:
data2

,text,label
0,i feel ashamed that i feel this way,0
1,i need to channel the anger i feel at these li...,0
2,i feel there are some aeronef fleets which are...,0
3,im in the relationship i still feel lonely,0
4,i feel victimized or stuck with a choice i hav...,0
...,...,...
83995,i have a feeling that we will be surprised wit...,5
83996,i feel overwhelmed with it when i eat too much...,5
83997,i left feel serence and impressed by the man h...,5
83998,i felt like i was even taking in more nicotine...,5


In [34]:
X=data2.drop('label',axis=1)

In [36]:
y=data2['label']

In [38]:
X.shape,y.shape

((84000, 1), (84000,))

In [40]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense

In [42]:
voc_size=100000

In [44]:
import nltk
import re
from nltk.corpus import stopwords

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranayyb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [47]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [48]:
corpus=[]
for i in range(0, len(X)):
    text = re.sub('[^a-zA-Z]', ' ', data2['text'][i])
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [49]:
len(corpus)

84000

In [51]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
len(onehot_repr)

84000

In [55]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)

In [56]:
embedded_docs.shape

(84000, 30)

In [57]:
from tensorflow.keras.layers import Dropout

In [58]:
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, GlobalMaxPooling1D

embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(500, return_sequences=True)))  # Reduced units for efficiency
model.add(Dropout(0.3))  # Reduced dropout rate
model.add(Bidirectional(LSTM(500, return_sequences=True)))
model.add(Dropout(0.3))  # Reduced dropout rate
model.add(GlobalMaxPooling1D())  # Added pooling layer
model.add(Dense(200, activation='relu'))  # Added dense layer before output
model.add(Dense(6, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-08-22 22:13:43.340162: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-08-22 22:13:43.340222: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-22 22:13:43.340227: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-22 22:13:43.340253: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-22 22:13:43.340276: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [60]:
len(embedded_docs),y.shape

(84000, (84000,))

In [61]:
embedded_docs[83564]

array([ 1070, 40253, 11115, 36331, 43274, 65773, 20829,  7476, 63953,
       97388, 94788,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [62]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=29)

In [64]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5)

Epoch 1/5


2024-08-22 22:13:44.413685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2100/2100 ━━━━━━━━━━━━━━━━━━━━ 592s 280ms/step - accuracy: 0.6849 - loss: 0.7420 - val_accuracy: 0.9448 - val_loss: 0.1495
Epoch 2/5
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 574s 273ms/step - accuracy: 0.9469 - loss: 0.1365 - val_accuracy: 0.9456 - val_loss: 0.1389
Epoch 3/5
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 568s 271ms/step - accuracy: 0.9517 - loss: 0.1174 - val_accuracy: 0.9464 - val_loss: 0.1389
Epoch 4/5
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 599s 285ms/step - accuracy: 0.9557 - loss: 0.1063 - val_accuracy: 0.9458 - val_loss: 0.1354
Epoch 5/5
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 626s 298ms/step - accuracy: 0.9564 - loss: 0.0983 - val_accuracy: 0.9388 - val_loss: 0.1497


In [77]:
y_pred=model.predict(X_test)

525/525 ━━━━━━━━━━━━━━━━━━━━ 46s 86ms/step


In [79]:
y_pred[0]

array([9.9203089e-06, 2.0589684e-06, 4.7183136e-08, 9.9991322e-01,
       7.4801952e-05, 8.1618614e-09], dtype=float32)

In [81]:
y_pred_classes = np.round(y_pred).astype(int)

In [83]:
y_pred_classes

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [85]:
y_test

array([3, 3, 2, ..., 0, 1, 3])

In [87]:
from sklearn.metrics import accuracy_score

if len(y_pred_classes.shape) > 1 and y_pred_classes.shape[1] > 1:
    y_pred_classes = np.argmax(y_pred_classes, axis=1)

if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy: {accuracy}')


Accuracy: 0.9374404761904762


In [107]:
def predict(text,model):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    # print(text)
    onehot_repr = [one_hot(text, voc_size)]
    embedded_text = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
    X_final = np.array(embedded_text)
    y_pred = model.predict(X_final)
    y_pred_classes = np.argmax(y_pred, axis=1)
    emotion=emotion_dict[y_pred_classes[0]]
    return emotion

In [109]:
predict("i feel devastated today",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


'sadness'

In [111]:
predict("it was a very good day",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


'joy'

In [113]:
predict("I don't know if I can do this. What if I fail?",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


'fear'

In [119]:
predict("I told you a million times to stop doing that!",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


'anger'

In [139]:
predict("I just got promoted! I'm so excited!",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'joy'

In [155]:
predict("his situation is just making me so angry!",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


'anger'

In [157]:
predict("i am soo happy you came",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'joy'

In [149]:
import pickle

In [151]:
with open('emotion.pkl', 'wb') as file:
    pickle.dump(model, file)

In [161]:
with open('emotion.pkl', 'rb') as file:
    loaded_model = pickle.load(file)